In [1]:
import requests
import pandas as pd

In [2]:
wiki_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
dfs=pd.read_html(wiki_url)

In [4]:
len(dfs)

3

In [5]:
dfs[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
df=dfs[0]

In [7]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
df1=df[df["Borough"]!="Not assigned"]
df1

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df1.loc[df1.Neighbourhood=="Not assigned"]

,Postal Code,Borough,Neighbourhood


In [10]:
df1.shape

(103, 3)

In [11]:
csv_url="http://cocl.us/Geospatial_data"
df_g=pd.read_csv(csv_url)
df_g.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_g.shape

(103, 3)

In [13]:
df1=df1.join(df_g.set_index("Postal Code"),on="Postal Code")
df1.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
# find out the latitude and longitude of Toronto
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
import folium
import numpy as np
map_toronto=folium.Map(location=[43.651070,-79.347015],zoom_start=10)
for lat, lng,borough,neighbourhood in zip(df1["Latitude"],df1["Longitude"],df1["Borough"],df1["Neighbourhood"]):
    label='{}{}'.format(neighbourhood,borough)
    label=folium.Popup(label,parser_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color='blue',fill=True,
                       fill_color="#3186cf",fill_opacity=0.7,
                       parse_html=False).add_to(map_toronto)
map_toronto.save("toronto_map.html")
map_toronto

In [45]:
CLIENT_ID="HH2HW3YU1E1GWY4SHPUNCFSDRY0110IDXSSKTSANBS52BO15"
CLIENT_SECRET="CKLI2LSMK0VXFHZACNC5OIAQMY5OLH3HX14FUBMRWVQYR4PC"
VERSION="20180605"
radius=1000
LIMIT=100
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        results=requests.get(url).json()["response"]["groups"][0]["items"]
        print(results)
        venues_list.append([(
            name,lat,lng,v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude',
                          'Venue','Venue Langitude','Venue Longitude','Venue Category']
    return(nearby_venues)
t_venues=getNearbyVenues(names=df1['Neighbourhood'],latitudes=df1['Latitude'],
                  longitudes=df1['Longitude'])


[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b', 'name': 'Brookbanks Park', 'location': {'address': 'Toronto', 'lat': 43.751976046055574, 'lng': -79.33214044722958, 'labeledLatLngs': [{'label': 'display', 'lat': 43.751976046055574, 'lng': -79.33214044722958}], 'distance': 245, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}, 'venuePage': {'id': '600917367'}}, 'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInte

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b16e8b6f964a52051bf23e3', 'name': 'Roots', 'location': {'address': '71-95 Orfus Road', 'lat': 43.71821373389962, 'lng': -79.46389305304558, 'labeledLatLngs': [{'label': 'display', 'lat': 43.71821373389962, 'lng': -79.46389305304558}], 'distance': 77, 'postalCode': 'M6A 1L9', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['71-95 Orfus Road', 'Toronto ON M6A 1L9', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d104951735', 'name': 'Boutique', 'pluralName': 'Boutiques', 'shortName': 'Boutique', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_boutique_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b16e8b6f964a52051bf23e3-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reaso

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b9d206bf964a520e69136e3', 'name': "Queen's Park", 'location': {'address': 'University Ave.', 'crossStreet': 'at Wellesley Ave.', 'lat': 43.66394609897775, 'lng': -79.39217952520835, 'labeledLatLngs': [{'label': 'display', 'lat': 43.66394609897775, 'lng': -79.39217952520835}], 'distance': 283, 'postalCode': 'M5R 2E8', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['University Ave. (at Wellesley Ave.)', 'Toronto ON M5R 2E8', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b9d206bf964a520e69136e3-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'T

[]
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': 'Wendy’s', 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'}]
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInter

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57eda381498ebe0e6ef40972', 'name': 'UNIQLO ユニクロ', 'location': {'address': '220 Yonge St', 'crossStreet': 'at Dundas St W', 'lat': 43.65591027779457, 'lng': -79.38064099181345, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65591027779457, 'lng': -79.38064099181345}], 'distance': 195, 'postalCode': 'M5B 2H1', 'cc': 'CA', 'neighborhood': 'Downtown Toronto', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['220 Yonge St (at Dundas St W)', 'Toronto ON M5B 2H1', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d103951735', 'name': 'Clothing Store', 'pluralName': 'Clothing Stores', 'shortName': 'Apparel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-57eda381498ebe0e6ef40972-0'}, {'reasons

...

[]
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b646a6ff964a5205cb12ae3', 'name': 'Cranberries', 'location': {'address': '601 Parliament St.', 'crossStreet': 'at Wellesley St. E', 'lat': 43.6678427705951, 'lng': -79.36940687874281, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6678427705951, 'lng': -79.36940687874281}], 'distance': 140, 'postalCode': 'M4X 1P9', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['601 Parliament St. (at Wellesley St. E)', 'Toronto ON M4X 1P9', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d147941735', 'name': 'Diner', 'pluralName': 'Diners', 'shortName': 'Diner', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b646a6ff964a5205cb12ae3-0'}, {'reasons': {'count': 0, 'items': [{'summary': 

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad4c05df964a52059f620e3', 'name': 'Canoe', 'location': {'address': '66 Wellington St West', 'crossStreet': 'at Bay Street', 'lat': 43.647452066183476, 'lng': -79.38132001815676, 'labeledLatLngs': [{'label': 'display', 'lat': 43.647452066183476, 'lng': -79.38132001815676}], 'distance': 133, 'postalCode': 'M5K 1H6', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['66 Wellington St West (at Bay Street)', 'Toronto ON M5K 1H6', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}, 'venuePage': {'id': '40552945'}}, 'referralId': 'e-0-4ad4c05df964a52059f620e3-0'}, {'reas

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4e3581d37d8b0c62b2d4e9e9', 'name': 'Lambton Kingsway Outdoor Pool', 'location': {'address': '15 Marquis Ave', 'lat': 43.65482054616071, 'lng': -79.50923055177938, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65482054616071, 'lng': -79.50923055177938}], 'distance': 225, 'postalCode': 'M8X 1W3', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['15 Marquis Ave', 'Toronto ON M8X 1W3', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d15e941735', 'name': 'Pool', 'pluralName': 'Pools', 'shortName': 'Pool', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4e3581d37d8b0c62b2d4e9e9-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '58d4650e2f91cb5ccec03447', 'name': 'Rorschach Brewing Co.', 'location': {'address': '1001 Eastern Ave', 'crossStreet': 'Woodfield Rd', 'lat': 43.6634831695922, 'lng': -79.31982368639481, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6634831695922, 'lng': -79.31982368639481}], 'distance': 162, 'postalCode': 'M4L 1A8', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['1001 Eastern Ave (Woodfield Rd)', 'Toronto ON M4L 1A8', 'Canada']}, 'categories': [{'id': '50327c8591d4c4b30a586d5d', 'name': 'Brewery', 'pluralName': 'Breweries', 'shortName': 'Brewery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-58d4650e2f91cb5ccec03447-0'}, {'reasons': {'count': 0, 'items': [{'summary': 

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4be0a6cf4f15c92829f6cb0b', 'name': 'Woodford Park', 'location': {'lat': 43.63315249748257, 'lng': -79.49626581098377, 'labeledLatLngs': [{'label': 'display', 'lat': 43.63315249748257, 'lng': -79.49626581098377}], 'distance': 390, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d1e8941735', 'name': 'Baseball Field', 'pluralName': 'Baseball Fields', 'shortName': 'Baseball Field', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/baseballfield_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4be0a6cf4f15c92829f6cb0b-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5afa07d0a92d98002c2174fa', 'name': 'Tornado 

In [46]:
t_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Langitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [47]:
t_venues_count=t_venues.groupby("Neighbourhood").count().reset_index()
t_venues_count[["Neighbourhood","Venue Category"]]

,Neighbourhood,Venue Category
0,Agincourt,5
1,"Alderwood, Long Branch",7
2,"Bathurst Manor, Wilson Heights, Downsview North",21
3,Bayview Village,4
4,"Bedford Park, Lawrence Manor East",22
...,...,...
91,"Willowdale, Willowdale West",5
92,Woburn,4
93,Woodbine Heights,9
94,York Mills West,2


In [48]:
print("There are {} unique categories.".format(len(t_venues['Venue Category'].unique())))

There are 277 unique categories.


In [49]:
t_venues_nor=pd.get_dummies(t_venues[["Venue Category"]],prefix="",prefix_sep="")
t_venues_nor["Neighbourhood"]=t_venues["Neighbourhood"]
f=[t_venues_nor.columns[-1]]+list(t_venues_nor.columns[:-1])
t_venues_nor=t_venues_nor[f]
t_venues_nor.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
t_venues_nor.shape

(2166, 278)

In [51]:
t_venues_grp=t_venues_nor.groupby("Neighbourhood").mean().reset_index()
t_venues_grp

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
t_venues_grp.shape

(96, 278)

In [53]:
num_top_venues=8
for hood in t_venues_grp["Neighbourhood"]:
    print("--------"+hood+"---------")
    tp=t_venues_grp[t_venues_grp["Neighbourhood"]==hood].T.reset_index()
    tp.columns=["Venue","freq"]
    tp=tp.iloc[1:]
    tp['freq']=tp['freq'].astype(float)
    tp=tp.round({'freq':2})
    print(tp.sort_values("freq",ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")
    


--------Agincourt---------
                       Venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2
5          Accessories Store   0.0
6        Moroccan Restaurant   0.0
7        Monument / Landmark   0.0


--------Alderwood, Long Branch---------
            Venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4     Coffee Shop  0.14
5        Pharmacy  0.14
6     Men's Store  0.00
7   Metro Station  0.00


--------Bathurst Manor, Wilson Heights, Downsview North---------
                       Venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                   Pharmacy  0.05
3  Middle Eastern Restaurant  0.05
4              Shopping Mall  0.05
5          Mobile Phone Shop  0.05
6        Fried Chicken Joint  0.05
7         Frozen Yogurt Shop  0.05


--------Bayview 

                     Venue  freq
0                      Bar  0.09
1              Coffee Shop  0.07
2                     Café  0.04
3              Men's Store  0.04
4    Vietnamese Restaurant  0.04
5               Restaurant  0.04
6         Asian Restaurant  0.04
7  New American Restaurant  0.02


--------Malvern, Rouge---------
                             Venue  freq
0             Fast Food Restaurant   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Moroccan Restaurant   0.0
4              Monument / Landmark   0.0
5  Molecular Gastronomy Restaurant   0.0
6       Modern European Restaurant   0.0
7                Mobile Phone Shop   0.0


--------Milliken, Agincourt North, Steeles East, L'Amoreaux East---------
                             Venue  freq
0                       Playground  0.33
1                           Bakery  0.33
2                             Park  0.33
3        Middle Eastern Restaurant  0.00
4              Monu

                             Venue  freq
0              Martial Arts School   1.0
1               Miscellaneous Shop   0.0
2                            Motel   0.0
3              Moroccan Restaurant   0.0
4              Monument / Landmark   0.0
5  Molecular Gastronomy Restaurant   0.0
6       Modern European Restaurant   0.0
7                Mobile Phone Shop   0.0




In [54]:
def return_most_common_venues(row,num_top_venues):
    rc=row.iloc[1:]
    rc_sorted=rc.sort_values(ascending=False)
    
    return rc_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues=10
indicators=['st','nd','rd']
columns=["Neighbourhood"]
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1,indicator[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))
nvenue_sorted=pd.DataFrame(columns=columns)
nvenue_sorted['Neighbourhood']=t_venues_grp['Neighbourhood']
for ind in np.arange(t_venues_grp.shape[0]):
    nvenue_sorted.iloc[ind,1:]=return_most_common_venues(t_venues_grp.iloc[ind,:],num_top_venues)
nvenue_sorted

,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Event Space,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,Greek Restaurant,Discount Store,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Ice Cream Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Middle Eastern Restaurant,Restaurant
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher
...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",Pizza Place,Grocery Store,Butcher,Coffee Shop,Pharmacy,Comic Shop,Diner,Ethiopian Restaurant,Escape Room,Electronics Store
92,Woburn,Coffee Shop,Mexican Restaurant,Korean BBQ Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
93,Woodbine Heights,Skating Rink,Curling Ice,Park,Intersection,Snack Place,Beer Store,Athletics & Sports,Dance Studio,Yoga Studio,Dog Run
94,York Mills West,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [56]:
from sklearn.cluster import KMeans
kclusters=4
tcl=t_venues_grp.drop("Neighbourhood",1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(tcl)
kmeans.labels_[:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 2, 3, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 0])

In [57]:
nvenue_sorted.insert(0,"Cluster Labels",kmeans.labels_)


In [58]:
tm=df1
tm.head()
tm=tm.join(nvenue_sorted.set_index("Neighbourhood"),on="Neighbourhood")

tm=tm.dropna()
tm.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Intersection,Construction & Landscaping,Diner,Ethiopian Restaurant,Escape Room
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Spa,Brewery,Shoe Store
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Airport Service
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Café,Restaurant,Chinese Restaurant,Portuguese Restaurant


In [59]:
import matplotlib.cm as cm
import matplotlib.colors as colors
mc=folium.Map(location=[latitude, longitude],zoom_start=10)
x=np.arange(kclusters)
ys=[i+x+(i*x) for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
markers_colors=[]
for lat,lon,poi,cluster in zip(tm['Latitude'],tm['Longitude'],tm['Neighbourhood'],tm["Cluster Labels"]):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],radius=5,popup=label,
    color=rainbow[int(cluster)-1],fill_opacity=0.7).add_to(mc)
mc

In [60]:
print("Cluster 1")
tm.loc[tm["Cluster Labels"]==0,tm.columns[[2]+list(range(6,tm.shape[1]))]].reset_index()

Cluster 1


,index,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Victoria Village,Pizza Place,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Intersection,Construction & Landscaping,Diner,Ethiopian Restaurant,Escape Room
1,4,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Spa,Brewery,Shoe Store
2,5,"Lawrence Manor, Lawrence Heights",Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Airport Service
3,6,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Café,Restaurant,Chinese Restaurant,Portuguese Restaurant
4,11,Don Mills,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Tennis Court,Sandwich Place,Bike Shop,Sporting Goods Shop,Café,Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
83,157,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Steakhouse,Asian Restaurant,Salad Place,American Restaurant
84,160,"The Kingsway, Montgomery Road, Old Mill North",River,Pool,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Drugstore
85,165,Church and Wellesley,Coffee Shop,Sushi Restaurant,Gay Bar,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Café,Men's Store,Yoga Studio,Hotel
86,168,"Business reply mail Processing Centre, South C...",Pizza Place,Auto Workshop,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio


In [61]:
print("Cluster 2")
tm.loc[tm["Cluster Labels"]==1,tm.columns[[2]+list(range(6,tm.shape[1]))]].reset_index()

Cluster 2


,index,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,89,"Humberlea, Emery",Baseball Field,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Discount Store
1,169,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Construction & Landscaping,Baseball Field,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [62]:
print("Cluster 3")
tm.loc[tm["Cluster Labels"]==2,tm.columns[[2]+list(range(6,tm.shape[1]))]].reset_index()

Cluster 3


,index,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [63]:
print("Cluster 4")
tm.loc[tm["Cluster Labels"]==3,tm.columns[[2]+list(range(6,tm.shape[1]))]].reset_index()

Cluster 4


,index,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Parkwoods,Park,Food & Drink Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,32,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
2,57,"East Toronto, Broadview North (Old East York)",Intersection,Park,Convenience Store,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,77,"North Park, Maple Leaf Park, Upwood Park",Bakery,Park,Construction & Landscaping,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,93,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Donut Shop,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Drugstore
5,98,Weston,Park,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
6,100,York Mills West,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,135,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Park,Bakery,Yoga Studio,Drugstore,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
8,147,Rosedale,Park,Trail,Playground,Tennis Court,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
